<a href="https://colab.research.google.com/github/nrwpradana/Predict-Serie-A-Match/blob/main/Scraping_Serie_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

In [ ]:
standings_url = "https://fbref.com/en/comps/11/Serie-A-Stats"

In [ ]:
data = requests.get(standings_url)

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [ ]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [ ]:
data = requests.get(team_urls[0])

In [ ]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [ ]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [ ]:
data = requests.get(f"https://fbref.com{links[0]}")

In [ ]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [ ]:
shooting.head()

For Napoli                                                       \
         Date   Time     Comp        Round  Day Venue Result GF GA   
0  2022-08-15  18:30  Serie A  Matchweek 1  Mon  Away      W  5  2   
1  2022-08-21  18:30  Serie A  Matchweek 2  Sun  Home      W  4  0   
2  2022-08-28  20:45  Serie A  Matchweek 3  Sun  Away      D  0  0   
3  2022-08-31  20:45  Serie A  Matchweek 4  Wed  Home      D  1  1   
4  2022-09-03  20:45  Serie A  Matchweek 5  Sat  Away      W  2  1   

                  ... Standard             Expected                            \
        Opponent  ...     Dist FK PK PKatt       xG npxG npxG/Sh G-xG np:G-xG   
0  Hellas Verona  ...     15.2  0  0     0      2.4  2.4    0.10  2.6     2.6   
1          Monza  ...     15.3  1  0     0      2.0  2.0    0.09  2.0     2.0   
2     Fiorentina  ...     14.7  1  0     0      1.7  1.7    0.13 -1.7    -1.7   
3          Lecce  ...     17.7  0  0     0      1.7  1.7    0.09 -0.7    -0.7   
4          Lazio  ...     16.1  0  0     0      2.1  2.1    0.11 -0.1    -0.1   

  Unnamed: 25_level_0  
         Match Report  
0        Match Report  
1        Match Report  
2        Match Report  
3        Match Report  
4        Match Report  

[5 rows x 26 columns]

In [ ]:
shooting.columns = shooting.columns.droplevel()

In [ ]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [ ]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2022-08-15,18:30,Serie A,Matchweek 1,Mon,Away,W,5.0,2.0,Hellas Verona,...,4-3-3,Michael Fabbri,Match Report,NaN,25,8,15.2,0,0,0
1,2022-08-21,18:30,Serie A,Matchweek 2,Sun,Home,W,4.0,0.0,Monza,...,4-3-3,Francesco Fourneau,Match Report,NaN,22,5,15.3,1,0,0
2,2022-08-28,20:45,Serie A,Matchweek 3,Sun,Away,D,0.0,0.0,Fiorentina,...,4-2-3-1,Livio Marinelli,Match Report,NaN,13,2,14.7,1,0,0
3,2022-08-31,20:45,Serie A,Matchweek 4,Wed,Home,D,1.0,1.0,Lecce,...,4-2-3-1,Matteo Marcenaro,Match Report,NaN,19,7,17.7,0,0,0
4,2022-09-03,20:45,Serie A,Matchweek 5,Sat,Away,W,2.0,1.0,Lazio,...,4-2-3-1,Simone Sozza,Match Report,NaN,19,7,16.1,0,0,0


In [ ]:
years = list(range(2022, 2019, -1))
all_matches = []

In [ ]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Serie A"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
len(all_matches)

60

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2022-08-15,18:30,Serie A,Matchweek 1,Mon,Away,W,5.0,2.0,Hellas Verona,...,Match Report,NaN,25.0,8.0,15.2,0.0,0,0,2022,Napoli
1,2022-08-21,18:30,Serie A,Matchweek 2,Sun,Home,W,4.0,0.0,Monza,...,Match Report,NaN,22.0,5.0,15.3,1.0,0,0,2022,Napoli
2,2022-08-28,20:45,Serie A,Matchweek 3,Sun,Away,D,0.0,0.0,Fiorentina,...,Match Report,NaN,13.0,2.0,14.7,1.0,0,0,2022,Napoli
3,2022-08-31,20:45,Serie A,Matchweek 4,Wed,Home,D,1.0,1.0,Lecce,...,Match Report,NaN,19.0,7.0,17.7,0.0,0,0,2022,Napoli
4,2022-09-03,20:45,Serie A,Matchweek 5,Sat,Away,W,2.0,1.0,Lazio,...,Match Report,NaN,19.0,7.0,16.1,0.0,0,0,2022,Napoli
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,2021-05-03,20:45,Serie A,Matchweek 34,Mon,Away,L,0,1,Torino,...,Match Report,NaN,10.0,1.0,18.1,0.0,0,0,2020,Parma
37,2021-05-09,15:00,Serie A,Matchweek 35,Sun,Home,L,2,5,Atalanta,...,Match Report,NaN,7.0,2.0,18.7,0.0,0,0,2020,Parma
38,2021-05-12,20:45,Serie A,Matchweek 36,Wed,Away,L,0,1,Lazio,...,Match Report,NaN,12.0,4.0,27.1,1.0,0,0,2020,Parma
39,2021-05-16,18:00,Serie A,Matchweek 37,Sun,Home,L,1,3,Sassuolo,...,Match Report,NaN,16.0,8.0,16.8,0.0,0,0,2020,Parma


In [ ]:
match_df.to_csv("matches_seriea.csv")